In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px

In [ ]:
"""
NOTE: combining 'ORI' and 'Incident Number' is not enough for a unique identifier per event. 
Waiting to hear clarification form Ben, but could create a key using date also. That should
be enough to join matching entries from one table to another correctly.
NOTE: Reply: heard back from Ben, got a new csv from a specific PD that only has 198 records, of which only 170 have unique INs 
"""

In [2]:
# Deliverables 1 & 2 Need to join:
# Data/2020 Domestic Violence - Administrative Data.csv
# Data/2020 Domestic Violence - Other Data.csv
# Data/2020 Q2 Domestic Violence - Location Type.csv
admin_df = pd.read_csv('../Data/2020 Domestic Violence - Administrative Data.csv', encoding='ISO-8859-1',skiprows=7)
other_df = pd.read_csv('../Data/2020 Domestic Violence - Other Data.csv', encoding='ISO-8859-1',skiprows=7)
location_df = pd.read_csv('../Data/2020 Q1 Domestic Violence - Location Type.csv', encoding='ISO-8859-1',skiprows=6)
#nadmin_df = pd.read_csv('../Data/new2020 Domestic Violence - Administrative for Athens PD 08192022.csv', encoding='ISO-8859-1',skiprows=6)


In [ ]:
#len(admin_df), len(admin_df.drop_duplicates()), len(set(admin_df['Incident Number']))

In [3]:
base_df = admin_df.drop_duplicates()
# MM/DD/YYYY -> need Day and Month specifically
base_df[['Month', 'Day','Year']] = base_df['Incident Date'].str.split('/', expand=True)
base_df[['Hour','End']] = base_df['Hour of Day'].str.split('-', expand=True)
defects = {
    '11:0 0am':'11:00am',
    '12:00n':'12:00pm',
    }
base_df.replace(to_replace=defects, inplace=True)
base_df.reset_index(drop=True, inplace=True)
# dropping records without time stamps as this would not contribute to a timeseries analysis
base_df = base_df[base_df.Hour != 'Unknown']
#base_df['Incident Date'] = pd.to_datetime(base_df['Incident Date'], format='%m/%d/%y')


/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_23633/2184331248.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df[['Month', 'Day','Year']] = base_df['Incident Date'].str.split('/', expand=True)
/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_23633/2184331248.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df[['Month', 'Day','Year']] = base_df['Incident Date'].str.split('/', expand=True)
/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_23633/2184331248.py:3

In [4]:
def change_time_format(x):
    in_time = datetime.strptime(x, "%I:%M%p")
    return datetime.strftime(in_time, "%H:%M")

def change_time_format2(x):
    n = int(x[:2])
    return n



In [5]:
base_df.Hour = base_df.Hour.apply(change_time_format)
base_df.Day = base_df.Day.apply(int)
base_df.Month = base_df.Month.apply(int)

/var/folders/pt/y3bb3b4j7yz3zzczhxc5l9p0vbrddc/T/ipykernel_23633/2486917750.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_df.Hour = base_df.Hour.apply(change_time_format)


In [6]:
ts_df = base_df[['ORI','Incident Number','Incident Date','Day','Month','Hour','Incident Clearance']]
ts_df.head()


,ORI,Incident Number,Incident Date,Day,Month,Hour,Incident Clearance
0,TN0410000,20010101-133,1/1/20,1,1,13:00,Not Cleared
1,TN0150100,202001000000.00,1/2/20,2,1,01:00,Cleared by Arrest
2,TN0150100,202001000000.00,1/2/20,2,1,11:00,Cleared by Arrest
3,TN0410000,200103-2142,1/3/20,3,1,21:00,Cleared by Arrest
4,TN0500100,20010166.00,1/2/20,2,1,11:00,Cleared by Arrest


In [ ]:
ts_df.dtypes

In [7]:
# puts the dates in order

days = list(set(ts_df['Incident Date']))
days.sort(key=lambda date: datetime.strptime(date, "%m/%d/%y"))

# creating frequency of events per day
per_day = {}
X = ts_df.groupby(['Incident Date'])['Incident Date'].count()
for i in days:
    per_day[i] = X[i]

# creating frequency of events per month
per_month = {}
Y = ts_df.groupby(['Month'])['Month'].count()
for m in Y.index:
    per_month[m] = Y[m]

# creating frequency of events per day of the month over the year
per_day_of_month = {}
Z = ts_df.groupby(['Day'])['Day'].count()
for d in Z.index:
    per_day_of_month[d] = Z[d]

# creating frequecy of overall events per hour 
per_hour_overall = {}
H = ts_df.groupby(['Hour'])['Hour'].count()
for h in H.index:
    per_hour_overall[h] = H[h]

# creating frequency by hours per month, [0] = per year, in desc order
# Ex. get top 3 hours for february hours_by_month[2][:3]
# Ex. want standard deviation for March hours_by_month[3].describe()['std']
hours_by_month = {}
days_per_month = {}
for i in range(13):
    if i == 0:
        # over all year
        hours_by_month[i] = ts_df.Hour.value_counts()
        days_per_month[i] = ts_df.Day.value_counts()
    else:
        # by month
        filter = ts_df[ts_df.Month == i]
        monthly = filter['Hour'].value_counts()
        daily = filter['Day'].value_counts()
        hours_by_month[i] = monthly
        days_per_month[i] = daily



In [ ]:
# computing monthly...
# hourly average (remember that there are 31 instances of each hour over march, so > day average makes sense)
print(hours_by_month[3][:1])
# daily average
print(days_per_month[1].mean())

In [8]:
# checking occurrences on January 1 against .loc
for j in range(1,13):

    for i in range(1,len(days_per_month[j])+1):
        n = days_per_month[j][i]
        t = ts_df.loc[ts_df.Month == j]['Day'].value_counts()[i]
        if n != t:
            print(f'Error at day {i}')

    print(f'Month {j} check complete.')

Month 1 check complete.
Month 2 check complete.
Month 3 check complete.
Month 4 check complete.
Month 5 check complete.
Month 6 check complete.
Month 7 check complete.
Month 8 check complete.
Month 9 check complete.
Month 10 check complete.
Month 11 check complete.
Month 12 check complete.


In [9]:
# getting means and std dev of days and hours per month
to_plot = {
    'Month':[],
    'Mean':[],
    'StDev':[],
    'Median':[],
    'Num Days':[]
}
for i in range(1,13):
    dpm = days_per_month[i].describe()
    # get the month's daily average & stdev 
    avg = round(dpm['mean'],2)
    std = round(dpm['std'],2)
    to_plot['Month'].append(i)
    to_plot['Mean'].append(avg)
    to_plot['StDev'].append(std)
    to_plot['Num Days'].append(dpm['count'])
    to_plot['Median'].append(dpm['50%'])

# pd.DataFrame(to_plot).plot.line(x='Month',y=['Mean','StDev'])


In [10]:
# boxplots per month
# trend line of mean and std per month
import plotly.graph_objects as go 


fig = go.Figure()
for i in range(1,13):
    date_object = datetime.strptime(str(i),'%m')
    mname = date_object.strftime('%b') 


    fig.add_trace(go.Box(
        y=days_per_month[i].values,
        x=[i]*len(days_per_month[i]),
        name=mname,
        marker_color='rgb(9,56,125)'
    ))

fig.update_layout(
    yaxis_title='Daily Instances of DA',
    title_text='Instances of DA per Day by Month'
)

fig.show()
fig.write_image('../Visuals/DA_Monthly_by_Day.png', engine="kaleido")

In [11]:
# creating frequency distribution per day of the month
day_df = pd.DataFrame(ts_df.groupby(['Day'])['Day'].count())
print(f"Average Incidents per day of the month: {ts_df.groupby(['Day'])['Day'].count().mean()}")
fig_day = px.bar(day_df, 
    x=day_df.index, 
    y='Day', 
    labels={'index':'Day of the Month', 'Day':'Count'},
    title='Incident Frequency Per Day of Month over the Year')
fig_day.show()

fig.write_image('../Visuals/Daily_total_over_year.png', engine="kaleido")

Average Incidents per day of the month: 1933.8387096774193


In [12]:
month_df = pd.DataFrame(ts_df.groupby(['Month'])['Month'].count())
print(f"Average Incidents per Month of the Year: {ts_df.groupby(['Month'])['Month'].count().mean()}")

fig_month = px.bar(month_df,
    x=month_df.index,
    y='Month',
    labels={'index':'Month','Month':'Count'},
    title='Total Incident Frequency Per Month')
fig_month.show()

fig.write_image('../Visuals/Monthly_total.png', engine="kaleido")

Average Incidents per Month of the Year: 4995.75


In [13]:
# scatter plot with x = hours, y = frequency, color by month
# creating data points in arrays
x = []
y = []
names = []
for i in range(0,24):
    # format hour
    if i < 10:
        ind = f'0{str(i)}:00'
    else:
        ind = f'{str(i)}:00'
    for j in range(1,13):
        date_object = datetime.strptime(str(j),'%m')
        names.append(date_object.strftime('%b'))

        x.append(ind)
        y.append(hours_by_month[j][ind])

df = pd.DataFrame({'Hour':x, 'Frequency': y, 'Month':names})

In [14]:


hourly_fig = px.scatter(df,
    x='Hour',
    y='Frequency',
    color='Month',  
    symbol='Month',
    # trendline='ols',
    # trendline_scope='overall' 
)

hourly_fig.update_layout(
    yaxis_title='Hourly Instances of DA',
    title_text='Instances of DA per Hour by Month'
)

hourly_fig.show()
hourly_fig.write_image('../Visuals/DA_Monthly_by_Hour.png', engine='kaleido')

In [ ]:
df2 = df
df2.Hour = df2.Hour.apply(change_time_format2)

In [27]:


overall_hourly_fig = px.scatter(df,
    x='Hour',
    y='Frequency',
    # color='Month',  
    # symbol='Month',
    trendline='ols',
    # trendline_options=dict(window=5)
)

overall_hourly_fig.update_layout(
    yaxis_title='Hourly Instances of DA',
    title_text='Overall DA per Hour (5-pt)'
)

overall_hourly_fig.show()
overall_hourly_fig.write_image('../Visuals/DA_Overall_by_Hour.png', engine='kaleido')